# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Feb 2 2023 8:43AM,256452,12,HH-39700,Hush Hush,Released
1,Feb 2 2023 8:43AM,256452,12,HH-39701,Hush Hush,Released
2,Feb 2 2023 8:43AM,256452,12,HH-39702,Hush Hush,Released
3,Feb 2 2023 8:43AM,256452,12,HH-39703,Hush Hush,Released
4,Feb 2 2023 8:43AM,256452,12,HH-39704,Hush Hush,Released
5,Feb 2 2023 8:43AM,256452,12,HH-39705,Hush Hush,Released
6,Feb 2 2023 8:43AM,256452,12,HH-39706,Hush Hush,Released
7,Feb 2 2023 8:43AM,256452,12,HH-39707,Hush Hush,Released
8,Feb 2 2023 8:43AM,256452,12,HH-39708,Hush Hush,Released
9,Feb 2 2023 8:43AM,256452,12,HH-39709,Hush Hush,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
35,256448,Released,29
36,256449,Released,27
37,256450,Released,1
38,256451,Released,21
39,256452,Released,14


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
256448,NaN,NaN,29.0
256449,NaN,NaN,27.0
256450,NaN,NaN,1.0
256451,NaN,NaN,21.0
256452,NaN,NaN,14.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
256370,6.0,2.0,30.0
256391,0.0,1.0,0.0
256392,0.0,0.0,1.0
256393,9.0,2.0,2.0
256401,0.0,1.0,0.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
256370,6,2,30
256391,0,1,0
256392,0,0,1
256393,9,2,2
256401,0,1,0


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,256370,6,2,30
1,256391,0,1,0
2,256392,0,0,1
3,256393,9,2,2
4,256401,0,1,0


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,256370,6,2,30
1,256391,,1,
2,256392,,,1
3,256393,9,2,2
4,256401,,1,


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Feb 2 2023 8:43AM,256452,12,Hush Hush
14,Feb 2 2023 8:34AM,256451,10,ISDIN Corporation
35,Feb 2 2023 8:33AM,256449,10,ISDIN Corporation
62,Feb 2 2023 8:33AM,256448,10,ISDIN Corporation
91,Feb 2 2023 8:31AM,256450,20,"Exact-Rx, Inc."
92,Feb 2 2023 8:30AM,256430,10,ISDIN Corporation
94,Feb 2 2023 8:30AM,256440,10,ISDIN Corporation
104,Feb 2 2023 8:21AM,256447,10,"Snap Medical Industries, LLC"
106,Feb 2 2023 8:21AM,256446,10,Emerginnova
107,Feb 2 2023 8:17AM,256445,19,Opus Life Sciences


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Feb 2 2023 8:43AM,256452,12,Hush Hush,,,14
1,Feb 2 2023 8:34AM,256451,10,ISDIN Corporation,,,21
2,Feb 2 2023 8:33AM,256449,10,ISDIN Corporation,,,27
3,Feb 2 2023 8:33AM,256448,10,ISDIN Corporation,,,29
4,Feb 2 2023 8:31AM,256450,20,"Exact-Rx, Inc.",,,1
5,Feb 2 2023 8:30AM,256430,10,ISDIN Corporation,2,,4
6,Feb 2 2023 8:30AM,256440,10,ISDIN Corporation,,,6
7,Feb 2 2023 8:21AM,256447,10,"Snap Medical Industries, LLC",,,2
8,Feb 2 2023 8:21AM,256446,10,Emerginnova,,,1
9,Feb 2 2023 8:17AM,256445,19,Opus Life Sciences,,,3


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 2 2023 8:43AM,256452,12,Hush Hush,14,,
1,Feb 2 2023 8:34AM,256451,10,ISDIN Corporation,21,,
2,Feb 2 2023 8:33AM,256449,10,ISDIN Corporation,27,,
3,Feb 2 2023 8:33AM,256448,10,ISDIN Corporation,29,,
4,Feb 2 2023 8:31AM,256450,20,"Exact-Rx, Inc.",1,,
5,Feb 2 2023 8:30AM,256430,10,ISDIN Corporation,4,,2
6,Feb 2 2023 8:30AM,256440,10,ISDIN Corporation,6,,
7,Feb 2 2023 8:21AM,256447,10,"Snap Medical Industries, LLC",2,,
8,Feb 2 2023 8:21AM,256446,10,Emerginnova,1,,
9,Feb 2 2023 8:17AM,256445,19,Opus Life Sciences,3,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 2 2023 8:43AM,256452,12,Hush Hush,14,,
1,Feb 2 2023 8:34AM,256451,10,ISDIN Corporation,21,,
2,Feb 2 2023 8:33AM,256449,10,ISDIN Corporation,27,,
3,Feb 2 2023 8:33AM,256448,10,ISDIN Corporation,29,,
4,Feb 2 2023 8:31AM,256450,20,"Exact-Rx, Inc.",1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 2 2023 8:43AM,256452,12,Hush Hush,14.0,NaN,NaN
1,Feb 2 2023 8:34AM,256451,10,ISDIN Corporation,21.0,NaN,NaN
2,Feb 2 2023 8:33AM,256449,10,ISDIN Corporation,27.0,NaN,NaN
3,Feb 2 2023 8:33AM,256448,10,ISDIN Corporation,29.0,NaN,NaN
4,Feb 2 2023 8:31AM,256450,20,"Exact-Rx, Inc.",1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 2 2023 8:43AM,256452,12,Hush Hush,14.0,0.0,0.0
1,Feb 2 2023 8:34AM,256451,10,ISDIN Corporation,21.0,0.0,0.0
2,Feb 2 2023 8:33AM,256449,10,ISDIN Corporation,27.0,0.0,0.0
3,Feb 2 2023 8:33AM,256448,10,ISDIN Corporation,29.0,0.0,0.0
4,Feb 2 2023 8:31AM,256450,20,"Exact-Rx, Inc.",1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,3077142,140.0,8.0,22.0
12,769267,15.0,1.0,0.0
16,256438,0.0,1.0,0.0
18,256436,1.0,0.0,0.0
19,1794973,16.0,5.0,16.0
20,512893,2.0,0.0,0.0
21,512783,1.0,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,3077142,140.0,8.0,22.0
1,12,769267,15.0,1.0,0.0
2,16,256438,0.0,1.0,0.0
3,18,256436,1.0,0.0,0.0
4,19,1794973,16.0,5.0,16.0
5,20,512893,2.0,0.0,0.0
6,21,512783,1.0,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,140.0,8.0,22.0
1,12,15.0,1.0,0.0
2,16,0.0,1.0,0.0
3,18,1.0,0.0,0.0
4,19,16.0,5.0,16.0
5,20,2.0,0.0,0.0
6,21,1.0,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,140.0
1,12,Released,15.0
2,16,Released,0.0
3,18,Released,1.0
4,19,Released,16.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,16,18,19,20,21
Status,,,,,,,
Completed,22.0,0.0,0.0,0.0,16.0,0.0,0.0
Executing,8.0,1.0,1.0,0.0,5.0,0.0,1.0
Released,140.0,15.0,0.0,1.0,16.0,2.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,16,18,19,20,21
0,Completed,22.0,0.0,0.0,0.0,16.0,0.0,0.0
1,Executing,8.0,1.0,1.0,0.0,5.0,0.0,1.0
2,Released,140.0,15.0,0.0,1.0,16.0,2.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,16,18,19,20,21
0,Completed,22.0,0.0,0.0,0.0,16.0,0.0,0.0
1,Executing,8.0,1.0,1.0,0.0,5.0,0.0,1.0
2,Released,140.0,15.0,0.0,1.0,16.0,2.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()